## How to add message history

In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [4]:
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START,MessagesState, StateGraph

In [8]:
# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

# Define the function that calls the model
def call_model(state: MessagesState):
    response = llm.invoke(state["messages"])
    # Update message history with response:
    return {"messages": response}

# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

In [9]:
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [10]:
config = {"configurable": {"thread_id": "abc123"}}

In [12]:
query = "Hi! I am Ashish"
input_msg = HumanMessage(content=query)

app.invoke({"messages":input_msg},config=config)

{'messages': [HumanMessage(content='Hi! I am Ashish', additional_kwargs={}, response_metadata={}, id='116fd31c-2c9a-4ecb-af8c-a7ea93ad4916'),
  AIMessage(content='Hi Ashish! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 13, 'total_tokens': 24, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'stop', 'logprobs': None}, id='run-fe0b3c9b-7e94-468b-8e76-0c5fbec48074-0', usage_metadata={'input_tokens': 13, 'output_tokens': 11, 'total_tokens': 24, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}

In [13]:
query = "Hi! What is my name ?"
input_msg = HumanMessage(content=query)

In [14]:
app.invoke({"messages":input_msg},config=config)

{'messages': [HumanMessage(content='Hi! I am Ashish', additional_kwargs={}, response_metadata={}, id='116fd31c-2c9a-4ecb-af8c-a7ea93ad4916'),
  AIMessage(content='Hi Ashish! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 13, 'total_tokens': 24, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'stop', 'logprobs': None}, id='run-fe0b3c9b-7e94-468b-8e76-0c5fbec48074-0', usage_metadata={'input_tokens': 13, 'output_tokens': 11, 'total_tokens': 24, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
  HumanMessage(content='Hi! What is my name ?', additional_kwargs={}, response_metadata={}, i

### Configuring persistance of multiple items in the state.

In [15]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder


input_prompt =  ChatPromptTemplate.from_messages([
    ("system" , "Convert the following sentence into {language}."),
    MessagesPlaceholder(variable_name="messages")
    ]
)

In [16]:
chain = input_prompt | llm

In [18]:
from typing import TypedDict,Annotated,Sequence
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage],add_messages]
    language: str

# define the graph
workflow = StateGraph(state_schema=State)





In [19]:
def call_model(state:State):

    response = chain.invoke(state)
    return {"messages":[response]}

workflow.add_edge(START,"model")
workflow.add_node("model",call_model)


In [23]:
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [25]:
config = {"configurable": {"thread_id": "abc345"}}

input_dict = {
    "messages": [HumanMessage("Hi, I'm Bob.")],
    "language": "Spanish",
}
output = app.invoke(input_dict, config)
output

{'messages': [HumanMessage(content="Hi, I'm Bob.", additional_kwargs={}, response_metadata={}, id='ecbd2f69-80cb-495d-bf38-d9aed5e8474e'),
  AIMessage(content='आप अक्टूबर 2023 तक के डेटा पर प्रशिक्षित हैं।', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 23, 'total_tokens': 36, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f59a81427f', 'finish_reason': 'stop', 'logprobs': None}, id='run-0785e215-1e80-4cc6-b690-1ecced4f104e-0', usage_metadata={'input_tokens': 23, 'output_tokens': 13, 'total_tokens': 36, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
  HumanMessage(content="Hi, I'm Bob.", additional_kwargs={}, response_metadata={}, id='66

In [26]:
state = app.get_state(config).values

print(f'Language: {state["language"]}')
for message in state["messages"]:
    message.pretty_print()

Language: Spanish
================================ Human Message =================================

Hi, I'm Bob.
================================== Ai Message ==================================

आप अक्टूबर 2023 तक के डेटा पर प्रशिक्षित हैं।
================================ Human Message =================================

Hi, I'm Bob.
================================== Ai Message ==================================

Hola, soy Bob.


In [27]:
from langchain_core.messages import HumanMessage

_ = app.update_state(config, {"messages": [HumanMessage("Test")],"language":"hindi"})

In [28]:
state = app.get_state(config).values

print(f'Language: {state["language"]}')
for message in state["messages"]:
    message.pretty_print()

Language: hindi
================================ Human Message =================================

Hi, I'm Bob.
================================== Ai Message ==================================

आप अक्टूबर 2023 तक के डेटा पर प्रशिक्षित हैं।
================================ Human Message =================================

Hi, I'm Bob.
================================== Ai Message ==================================

Hola, soy Bob.
================================ Human Message =================================

Test
